#DATASCI W261: Machine Learning at Scale

#Vineet Gangwar
**vineet.gangwar@gmail.com  
W261-2: Machine Learning at Scale  
Assignment #4  
Date: Sep - 29 - 2015**

#HW 4.0. 
What is MrJob? How is it different to Hadoop MapReduce?  
What are the mapper_final(), combiner_final(), reducer_final() methods? When are they called?

**Answer**  
- *What is MRJob?*  
MRJob is a python module, which is developed by Yelp, that provides a Python environment in which to develop, test and run MapReduce jobs.  
  
  
- *How is it different to Hadoop MapReduce?*  
MRJob can run locally, on a hadoop cluster and on Amazon EMR. To use python as map and reduce tasks in Hadoop, we use Hadoop streaming to communicate with Hadoop via stdin and stdout. While in MRJob each MapReduce job is defined as a class and the Map and reduce tasks are defined as its methods. MRJob can handle complex input/output via serialization while hadoop can only handle text as input/output. MRJob makes the output values of the mapper available to the reducer as a generator object, while Hadoop sends the key/value pairs line by line via stdin  
  
  
- *What are the mapper_final(), combiner_final(), reducer_final() methods? When are they called?*  
Mapper_final, combiner_final and reducer_final are methods provided by MRJob that helps in code organization, housekeeping and optimization. Mapper_final is called after the mapper method reaches it end of input. Similarly, combiner_final and reducer_final are called after the combiner and reducer mthods reach the end of their respective inputs

#HW 4.1
What is serialization in the context of MrJob or Hadoop?  
When it used in these frameworks?  
What is the default serialization mode for input and outputs for MrJob?  

**Answer**  

- *What is serialization in the context of MrJob or Hadoop?*  
  
Serialization enables MRJob to pass values (simple and complex) between the tasks of a job. This is necessary because messages between mappers and reducers have to mostly travel over the network. Serialization is the process in which an object is converted into a byte stream that can be sent over the network. De-serialization is the process by which the byte stream is re-converted back into python objects

- *When it used in these frameworks?*  
  
It is used by whenever data is exchanged between tasks of a MapReduce job and between the Hadoop framework and MRJob. Mappers, combiners, and reducers are examples of tasks
  
- *What is the default serialization mode for input and outputs for MrJob?*  
  
Default mode for input = RawValueProtocol  
Default mode for output = JSONProtocol

#HW 4.2: 
Recall the Microsoft logfiles data from the async lecture. The logfiles described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html  
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:
  
C,"10001",10001   #Visitor id 10001  
V,1000,1          #Visit by Visitor 10001 to page id 1000  
V,1001,1          #Visit by Visitor 10001 to page id 1001  
V,1002,1          #Visit by Visitor 10001 to page id 1002  
C,"10002",10002   #Visitor id 10001  
V  
Note: #denotes comments  
to the format:  
  
V,1000,1,C, 10001  
V,1001,1,C, 10001  
V,1002,1,C, 10001  
  
Write the python code to accomplish this.  

**Answer**  
The code below reads the input file sequentially. Whenever a line containing customer information is found, it is stored in a string. This string is appended to all subsequent lines containing vroot information and printed to a file. The process continues until the end of input is reached

In [3]:
input_filename = 'anonymous-msweb.data'
output_filename = 'msweb_processed'

output_filehndl = open(output_filename, 'w')

current_customer = str()
output_line = str()

with open(input_filename, 'r') as input_filehndl:
    for i, line in enumerate(input_filehndl):
        fields = line.strip().split(',')
        if fields[0] == 'C':
            current_customer = fields[0] + ',' + fields[2]
        if fields[0] == 'V':
            output_line = line.strip() + ',' + current_customer + '\n'
            output_filehndl.write(output_line)
output_filehndl.close()

#HW 4.3
Find the 5 most frequently visited pages using mrjob from the output of 4.2 (i.e., transfromed log file).

**Answer**  
  
*Mapper*  
The mapper splits each line of the input and emits the vroot id and the digit 1.
  
*Reducer*  
The reducer sums up the counts of each vroot and stored in this information as list of tuples  
  
*Reducer_final*  
Reducer_final reverse sorts the stored list of tuples by visit count and emits the top 5 vroot ids and their respective counts  
  
The output is in the format: Vroot_id, Visit_Count

In [6]:
%%writefile frequent_pages_top5.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import operator

class FrequentPages(MRJob):
    def mapper(self, line_no, line):
        # Extracts the Vroot that was visited
        fields = line.strip().split(',')
        if fields[0] == 'V':
            yield fields[1], 1

    def reducer_init(self):
        self.vroot_list = list()
    
    def reducer(self, vroot, visit_counts):
        # Sums Vroot visit counts
        total = sum(visit_counts)
        self.vroot_list.append((vroot, total))
    
    def reducer_final(self):
        # Sorting vroot_list based on frequency
        self.vroot_list = sorted(self.vroot_list, key=operator.itemgetter(1), reverse=True)
        for item in self.vroot_list[:5]:
            yield item
        
    def steps(self):
        return [
            MRStep(mapper        = self.mapper,
                   reducer_init  = self.reducer_init,
                   reducer       = self.reducer,
                   reducer_final = self.reducer_final)
        ]


Overwriting frequent_pages_top5.py


In [7]:
import imp
import frequent_pages_top5
imp.reload(frequent_pages_top5)

mr_job = frequent_pages_top5.FrequentPages(args=['msweb_processed'])

with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

(u'1008', 10836)
(u'1034', 9383)
(u'1004', 8463)
(u'1018', 5330)
(u'1017', 5108)


#HW 4.4
Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

**Answer**  
To reduce output clutter, I am formatted the output in the following:  
vroot1_id URL  
&nbsp;&nbsp;&nbsp;&nbsp;customer1 visit_count  
&nbsp;&nbsp;&nbsp;&nbsp;customer2 visit_count  
&nbsp;&nbsp;&nbsp;&nbsp;...  
vroot2_id URL  
&nbsp;&nbsp;&nbsp;&nbsp;customer1 visit_count  
&nbsp;&nbsp;&nbsp;&nbsp;customer2 visit_count  
&nbsp;&nbsp;&nbsp;&nbsp;...  
...  
The question did not sepcify the output format, but it is a matter one line that will result in the output in the format  
vroot1_id URL customer1 visit_count  
vroot1_id URL customer2 visit_count  
vroot2_id URL customer1 visit_count  
vroot2_id URL customer2 visit_count  
...  
Additonally, since the output is very big so I have stored the output into a file and displayed head 50 and tail 50 line

**Mapper**  
The mapper splits the input lines and emits vroot_id and the digit 1  
  
**Reducer_init**  
Reads in the lines starting with A from the original file and stored vroot_id and url in a dictionary. This dictionary is used by the reducer get url associated with vroot_ids  
  
**Reducer**  
For each key (i.e. vroot_id), the reducer creates a dictionary of customer_id and customer_visit_count mapping.  
Then the reducer, uses the dictionary created in reducer_init to obtain the url associated with a vroot_id.  
Then it emits all customer_ids and associated visit_counts where the visit_count is maximum

In [4]:
%%writefile frequent_visitor_per_page.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class FrequentVisitors(MRJob):    
    def mapper(self, line_no, line):
        # Extracts the Vroot and customer id
        fields = line.strip().split(',')
        yield fields[1], fields[4]
        
    def reducer_init(self):
        # Initializing dict for storing customer counts per vroot
        self.cust_count_per_vroot = dict()
        # Creating dict for vroot_id and url mapping
        self.vroot_url = dict()
        filename = '/tmp/anonymous-msweb.data'
        with open(filename, 'r') as filehndl:
            for line in filehndl:
                fields = line.strip().split(',')
                if fields[0] == 'A':
                    vroot = fields[1]
                    url = fields[4].replace('"', '')
                    self.vroot_url[vroot] = url
    
    def reducer(self, vroot, cust_list):
        # For each key creating dict of customer_id and visit_count mapping
        for cust in cust_list:
            self.cust_count_per_vroot.setdefault(cust, 0)
            self.cust_count_per_vroot[cust] += 1
        # Emit vroot and url
        yield vroot, self.vroot_url[vroot]
        # Emit cust ids with max count in the dict self.cust_count_per_vroot
        max_count = max(self.cust_count_per_vroot.values())
        for k, v in self.cust_count_per_vroot.iteritems():
            if v == max_count:
                # Adding a little format for display as per homework requirements
                yield "    " + k, str(v)
        # Re-initializing dict
        self.cust_count_per_vroot = dict()
        
    def steps(self):
        return [
            MRStep(mapper       = self.mapper,
                   reducer_init = self.reducer_init,
                   reducer      = self.reducer)
        ]

Overwriting frequent_visitor_per_page.py


In [5]:
import imp
import frequent_visitor_per_page
imp.reload(frequent_visitor_per_page)

# Copying file anonymous-msweb.data to /tmp
!cp anonymous-msweb.data /tmp/

output_filename = 'output'
filehndl = open(output_filename, 'w')
mr_job = frequent_visitor_per_page.FrequentVisitors(args=['msweb_processed'])

with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        # Writing job output into a file
        outline = " ".join(mr_job.parse_output_line(line))
        filehndl.write(outline + '\n')
filehndl.close()

# Displaying head -n 50
print ">> Displaying head -n 50"
!head -n 50 output
# Displaying tail -n 50
print "\n>> Displaying tail -n 50"
!tail -n 50 output

>> Displaying head -n 50
1000 /regwiz
    36585 1
    20914 1
    35230 1
    16073 1
    35549 1
    31899 1
    33730 1
    18882 1
    38670 1
    29358 1
    28996 1
    10457 1
    11544 1
    10454 1
    22667 1
    10511 1
    10101 1
    30428 1
    15204 1
    18950 1
    16427 1
    23697 1
    13085 1
    37596 1
    23049 1
    13080 1
    33749 1
    18420 1
    18421 1
    33425 1
    29086 1
    19409 1
    40152 1
    33037 1
    13248 1
    18817 1
    28109 1
    22760 1
    32654 1
    12930 1
    19376 1
    33924 1
    22765 1
    41651 1
    16676 1
    40702 1
    40703 1
    42458 1
    12700 1

>> Displaying tail -n 50
    31810 1
    21468 1
    33561 1
    40758 1
    37140 1
    29417 1
    38571 1
    21966 1
    21622 1
    20717 1
    42385 1
    40390 1
    25564 1
    39900 1
    16980 1
    40679 1
    29674 1
    25846 1
    12297 1
    21816 1
    36955 1
    11721 1
    31229 1
    31635 1
    30082 1
    17192 1
    38981 1
    34785 1
    14002 1


#HW 4.5 
Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342  
http://arxiv.org/abs/1508.01843  

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words  
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution   
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution   
(D) K=4 "trained" centroids, determined by the sums across the classes.  

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

**Answer**  
###Results
Class distribution per cluster  
4.5 (A): Uniform Random K=4

Number of Iterations: **15**


|        |   0 |  1 |  2 |  3 | total |
|--------|-----|----|----|----|-------|
|cluster |     |    |    |    |       |
|0       |   1 | 82 | 34 |  4 |   121 |
|1       | 171 |  2 | 17 | 27 |   217 |
|2       |   0 |  6 |  3 |  0 |     9 |
|3       | 580 |  1 |  0 | 72 |   653 |

4.5 (B): Perturbed Random K=2

Number of Iterations: **4**

|        |   0 |  1 |  2 |  3 | total |
|--------|-----|----|----|----|-------|
|cluster |     |    |    |    |       |
|0       | 751 |  3 | 16 | 99 |   869 |
|1       |   1 | 88 | 38 |  4 |   131 |
|2       |   0 |  0 |  0 |  0 |     0 |
|3       |   0 |  0 |  0 |  0 |     0 |
4.5 (C) Perturbed Random K=4

Number of Iterations: **8**

|        |  0  | 1  | 2  | 3  |total  |
|--------|-----|----|----|----|-------|
|cluster |     |    |    |    |       |
|0       | 93  | 2  |14  |57  |  166  |
|1       |658  | 1  | 0  |42  |  701  |
|2       |  0  |51  | 2  | 0  |   53  |
|3       |  1  |37  |38  | 4  |   80  |

4.5 (D) Trained K=4

Number of Iterations: **5**

|        | 0  | 1  | 2  | 3  |total  |
|--------|----|----|----|----|-------|
|cluster |    |    |    |    |       |
|0       |749 |  3 | 14 | 38 |   804 |
|1       |  0 | 51 |  0 |  0 |    51 |
|2       |  1 | 37 | 40 |  4 |    82 |
|3       |  2 |  0 |  0 | 61 |    63 |

###Discussion of results
Trained centroids did converge fast but did not result in perfect clustering. When K=4, perturbed random provided better (closer to trained centroids) and faster results than uniform random. Perturbed random with K=2 was the fasted but that's probably because the centroids were far enough. From the above results, perturbed random seems to be the best option for selecting centroids. It will be interesting to check if Kmeans++ is a better algorithm at choosing seed centroids than perturbed random


**Code Description**  
&nbsp;&nbsp;&nbsp;&nbsp;**Mapper**  
The mapper is helped by the mapper_init method and the two other help functions. The mapper does three things:  
- Normalizes each observation by the respective sum of words
- Uses an helper function to find the index of the closest centroid
- Emits centroid index and a comman separated string containing the observation
  
Mapper helper functions:  
- Mapper_init:     Reads in centroids from an external file as stores in nd.array
- Distance:        Returns the Euclidean distance between two 1000 points. (It can handle any number of dimensions)
- Closest_Centroid: Given centroids and an observation, it returns of the index of the closest centroid to the observation
  
&nbsp;&nbsp;&nbsp;&nbsp;**Reducer**  
The reducer receives a centroid index as key and all observations that are closet to it. It does the following:  
- Store all the observations in a nd.array
- Yield the index and the number of observations (This shows the change in the number of observations per cluster per iteration)
- Calculates new centroids as the mean of the observations per cluster
- Use Reducer_final method to write the new centroids back into the centroids file
  
Reducer helper functions:  
- Reducer_init:    Reads centroids from a file and makes it available to the reducer
- Reducer_final:   Writes back the newly calculated centroids into the file

In [36]:
%%writefile kmeans_twitter.py

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
import numpy as np

def distance(np_data1, np_data2):
    # Calculate Euclidean distance between two points in 1000 dimensional space
    diff = np_data1 - np_data2
    diff_sq = diff**2
    diff_sq_sum = sum(diff_sq)
    dist = diff_sq_sum**0.5
    return dist

def closest_centroid(centroids, observation):
    # Given centroids and an observation, this function returns the index of the closet centroid to the observation
    # Number of centroids
    num_centroids = len(centroids)
    # Initializing distance array that will hold distances of the observation to the centroids
    np.distances = np.zeros(num_centroids)
    # Updating distance array
    for i, centroid in enumerate(centroids):
        np.distances[i] = distance(centroid, observation)
    # return closet centroid's index using argmin
    return np.argmin(np.distances)

class kmeans(MRJob):
    def mapper_init(self):
        # Reading centroids and storing this as numpy nd.array
        filename = '/tmp/centroids'
        self.centroids = list()
        with open(filename, 'r') as filehndl:
            for line in filehndl:
                fields = line.strip().split(',')
                np_fields = np.array(fields).astype(float)
                self.centroids.append(np_fields)
        self.centroids = np.array(self.centroids)
    
    def mapper(self, line_no, line):
        # Gets a observation at a time
        # 1st step: Normalize the observation using sum of words
        # 2nd Step: Find the index of the closet centroid
        # 3rd Step: Emit the index and the observation as a comman separated string
        fields = line.strip().split(',')
        user_id = fields[0]
        user_class = fields[1]
        sum_words = float(fields[2])
        all_words = fields[3:]
        np_all_words = np.array(all_words).astype(float) / sum_words
        nearest_centroid_idx = closest_centroid(self.centroids, np_all_words)
        #print "np_all_words", ','.join(np_all_words.astype(str))
        yield nearest_centroid_idx, ','.join(np_all_words.astype(str))
    
    def reducer_init(self):
        # Stores all the observations/members for a class. It is used to calculate the new centroid by calculating the means
        self.class_members = list()
        
        # Creating variable to store new_centroids that will be created by the reducer
        self.new_centroids = list()
        # Initializing self.new_centroids with initial values from the centroids file
        filename = '/tmp/centroids'
        with open(filename, 'r') as filehndl:
            for line in filehndl:
                fields = line.strip().split(',')
                np_fields = np.array(fields).astype(float)
                self.new_centroids.append(np_fields)
        self.new_centroids = np.array(self.new_centroids)
    
    def reducer(self, idx, members):
        # Read all the observations/members of a class and store it in nd.array
        for member in members:
            member_list = member.split(',')
            self.class_members.append(member_list)
        np_class_members = np.array(self.class_members).astype(float)
        
        # yield cluster index and number of members
        yield idx, np_class_members.shape[0]
        
        # Finding mean of members i.e. new centroid
        new_centroid = np.mean(np_class_members, axis=0)
        # Updating new centroid into variable initialized in Reducer_init
        self.new_centroids[idx] = new_centroid
        # Resetting self.class_members
        self.class_members = list()
    
    def reducer_final(self):
        # Write new_centroids into file
        filename = '/tmp/centroids'
        filehndl = open(filename, 'w')
        for i in self.new_centroids:
            centroid = ','.join(i.astype(str))
            filehndl.write(centroid + '\n')
        filehndl.close()
        
    def steps(self):
        return [
            MRStep(mapper_init   = self.mapper_init,
                   mapper        = self.mapper,
                   reducer_init  = self.reducer_init,
                   reducer       = self.reducer,
                   reducer_final = self.reducer_final)
        ]

Overwriting kmeans_twitter.py


##Helper functions to obtain seed centroids  
The functions in the cell below are used to calculate seed centroids.  

- K=4 Uniform random:
 1. Read_input_into_ndarray(): Reads and normalizes topUsers_Apr-Jul_2014_1000-words.txt into nd.array
 2. uniform_random_centroids(): Randomly selects centroids from the above nd.array
- K=2 or 4 Perturbed random:
 1. Dataset_Centroid(): Calculates and returns the centroid of the entire dataset
 2. Get_perturbed_centroids(): Perturbs the dataset centroid and returns 2 or 4 new centroids
- K=4 Trained:
 1. Get_trainined_centroids(): Returns normalized mean of the sum of all labeled features

In [60]:
# Functions to get centroids
import numpy as np

# Getting uniform random Centroids
# ==========
# Reading input file topUsers_Apr-Jul_2014_1000-words.txt into memory
# Returns normalized nd.array of the observations
def read_input_into_ndarray(filename):
    data_list = list()
    with open(filename, 'r') as filehndl:
        for line in filehndl:
            fields = line.strip().split(',')
            user_id = fields[0]
            user_class = fields[1]
            sum_words = float(fields[2])
            all_words = fields[3:]
            np_all_words = np.array(all_words).astype(float) / sum_words
            data_list.append(np_all_words)
    np_data_list = np.array(data_list)
    return np_data_list

# Choosing seed uniform centroids randomly from the data
def uniform_random_centroids(np_data_list, num_centroids):
    centroids = list()
    for i in range(np_data_list.shape[1]):
        column = np_data_list[:,i]
        choice = np.random.choice(column, size=num_centroids, replace=True)
        centroids.append(choice)
    np_centroids = np.array(centroids)
    np_centroids = np_centroids.transpose()
    return np_centroids
# ==========

# Getting Perturbed Centroids
# ==========
def Dataset_Centroid():
    # Reading aggregrate values of the dataset and return dataset centroid
    filename = 'topUsers_Apr-Jul_2014_1000-words_summaries.txt'
    with open(filename, 'r') as filehndl:
        for line in filehndl:
            fields = line.strip().split(',')
            if fields[0] == 'ALL_CODES':
                sum_all_words = float(fields[2])
                all_words = fields[3:]
                # finding centroids of dataset
                dataset_centroid = np.array(all_words).astype(float) / sum_all_words
    return dataset_centroid

def get_perturbed_centroids(dataset_centroid, k):
    # Generating perturbed centroids
    perturbations = np.random.normal(0, 0.0001, (k, 1000))
    new_centroids = perturbations + dataset_centroid
    return new_centroids
# ==========

# Getting Trained Centroids
# ==========
def get_trainined_centroids():
    # Reading aggregrate values of the dataset
    centroids = list()
    filename = 'topUsers_Apr-Jul_2014_1000-words_summaries.txt'
    with open(filename, 'r') as filehndl:
        for line in filehndl:
            fields = line.strip().split(',')
            if fields[0] == 'CODE':
                sum_all_words = float(fields[2])
                all_words = fields[3:]
                class_centroid = np.array(all_words).astype(float) / sum_all_words
                centroids.append(class_centroid)
    centroids = np.array(centroids)
    return centroids
# ==========

## Additional helper functions
- write_to_centroids_to_file(): Writes an nd.array containing centroids to file
- read_centroids_file(): Reads a file containing centroids and returns a flattened one dimensional result
- stop_criterion(): Stopping criteria to stop the algorithm. All dimensions of all successive centroids below a preset threshold

In [38]:
# Other supporting functions

# Writing Centroids into file
def write_to_centroids_to_file(centroids):
    filename = '/tmp/centroids'
    filehndl = open(filename, 'w')
    for i in centroids:
        centroid = ','.join(i.astype(str))
        filehndl.write(centroid + '\n')
    filehndl.close()
# -----------

# To read a flattened centroids file
def read_centroids_file():
    filename = '/tmp/centroids'
    filetxt = list()
    with open(filename, 'r') as filehndl:
        for line in filehndl:
            line = line.strip().split(',')
            filetxt.append(line)
    flattened = np.array(filetxt).flatten().astype(float)
    return flattened

# Stopping criteria
# Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new, t):
    diff = centroid_points_old - centroid_points_new
    diff = np.absolute(diff)
    flag = True
    for i in diff:
        if(i>t):
            flag = False
            break
    return flag


##4.5(A)
###K=4 uniform random centroid-distributions over the 1000 words
It follows the following steps:
- Generate Centroids by using helper function described above
- Write centroids into file
- Run MapReduce job with an upper limit of 1000 iterations
- Prints number of members of each cluster after each iteration
- Stops after the stopping criteria (described above) threshold of 0.001 is breached

In [39]:
# MapReduce for Uniform Random seed centroids

import imp
from itertools import chain
import kmeans_twitter
imp.reload(kmeans_twitter)

input_file_name = 'topUsers_Apr-Jul_2014_1000-words.txt'
mr_job = kmeans_twitter.kmeans(args=[input_file_name])

# Getting Centroids
num_centroids = 4
np_data_list = read_input_into_ndarray(input_file_name)
centroids = uniform_random_centroids(np_data_list, num_centroids)
print "Seed Centroids:"
print centroids

# Write Centroid to file
write_to_centroids_to_file(centroids)

# Running MapReduce twice with overall iteration limit set to 1000
for i in range(1000):
    print ">> Iteration:", i+1
    print ">> (Assigned Cluster, Number of Obserations)"
    old_centroids = read_centroids_file()
    # Running MapReduce job
    with mr_job.make_runner() as runner:
        runner.run()
        # Capturing MapReduce job output and printing to console
        for line in runner.stream_output():
            print mr_job.parse_output_line(line)
    new_centroids = read_centroids_file()
    if stop_criterion(old_centroids, new_centroids, 0.001):
        print "Stopping Criteria Reached"
        break
        

Seed Centroids:
[[  2.47728442e-02   8.58461208e-04   3.51624933e-02 ...,   0.00000000e+00
    2.02691746e-05   3.34552059e-04]
 [  5.38851169e-03   4.87462443e-02   2.42808129e-02 ...,   0.00000000e+00
    1.95151568e-04   1.18536782e-04]
 [  2.41937330e-02   2.58005044e-02   2.09809479e-02 ...,   0.00000000e+00
    9.45596671e-05   6.49878148e-05]
 [  9.91673683e-03   5.26544147e-02   1.22710528e-02 ...,   0.00000000e+00
    2.84123196e-05   0.00000000e+00]]
>> Iteration: 1
>> (Assigned Cluster, Number of Obserations)
(0, 156)
(1, 20)
(2, 2)
(3, 822)
>> Iteration: 2
>> (Assigned Cluster, Number of Obserations)
(0, 134)
(1, 176)
(2, 3)
(3, 687)
>> Iteration: 3
>> (Assigned Cluster, Number of Obserations)
(0, 123)
(1, 238)
(2, 8)
(3, 631)
>> Iteration: 4
>> (Assigned Cluster, Number of Obserations)
(0, 121)
(1, 273)
(2, 10)
(3, 596)
>> Iteration: 5
>> (Assigned Cluster, Number of Obserations)
(0, 122)
(1, 295)
(2, 9)
(3, 574)
>> Iteration: 6
>> (Assigned Cluster, Number of Obserations)

##Helper functions for summarizing results
- Distance(): Returns the Euclidean distance between two 1000 points. (It can handle any number of dimensions)
- Closest_Centroid(): Given centroids and an observation, it returns of the index of the closest centroid to the observation
- Read_final_centroids(): Reads the final centroids file generated by Kmeans and returns it in an nd.array
- input_data_generator(): This function produces a generator. It generates all the 1000 normalized observations in the original dataset along with their label
- results_data_structure(): Returns a 4 X 4 pandas DataFrame that will store the class break-up per cluster

In [40]:
# Support functions for summarizing clustering results  

import numpy as np
import pandas as pd

def distance(np_data1, np_data2):
    # Calculate Euclidean distance between two points in 1000 dimensional space
    diff = np_data1 - np_data2
    diff_sq = diff**2
    diff_sq_sum = sum(diff_sq)
    dist = diff_sq_sum**0.5
    return dist

def closest_centroid(centroids, observation):
    # Number of centroids
    num_centroids = len(centroids)
    # Initializing distance array
    np.distances = np.zeros(num_centroids)
    # Updating distance array
    for i, centroid in enumerate(centroids):
        np.distances[i] = distance(centroid, observation)
    #print np.distances
    return np.argmin(np.distances)

# Reading final cluster file
def read_final_centroids():
    filename = '/tmp/centroids'
    centroids = list()
    with open(filename, 'r') as filehndl:
        for line in filehndl:
            fields = line.strip().split(',')
            np_fields = np.array(fields).astype(float)
            centroids.append(np_fields)
    centroids = np.array(centroids)
    return centroids

# Input data generator
# Yields label and normalized observation data for each entry in the original dataset
def input_data_generator():
    filename = 'topUsers_Apr-Jul_2014_1000-words.txt'
    with open(filename, 'r') as filehndl:
        for line in filehndl:    
            fields = line.strip().split(',')
            user_id = fields[0]
            user_class = fields[1]
            sum_words = float(fields[2])
            all_words = fields[3:]
            np_all_words = np.array(all_words).astype(float) / sum_words
            yield (user_class, np_all_words)

# Initialize final result data structure
def results_data_structure():
    results = dict()
    results['cluster'] = [0, 1, 2, 3]
    results['total'] = [0 for i in range(4)]
    results[0] = [0 for i in range(4)]
    results[1] = [0 for i in range(4)]
    results[2] = [0 for i in range(4)]
    results[3] = [0 for i in range(4)]
    results = pd.DataFrame(results)
    results.set_index('cluster', inplace=True)
    return results



## The code below generates the class break-up per cluster
The rows are the clusters as assigned by Kmeans.  
The columns are the number of observations per label

In [41]:
# Generating summary results for uniform random centroids
centroids = read_final_centroids()
results = results_data_structure()

# Looping through Input data
for c, d in input_data_generator():
    assigned_centroid = closest_centroid(centroids, d)
    results.loc[assigned_centroid, 'total'] += 1
    results.loc[assigned_centroid, int(c)] += 1

print ">> Cluster Summary"
print ">> Rows: Assigned Cluster; Columns: Actual Classes"
print "\n"
print results

>> Cluster Summary
>> Rows: Assigned Cluster; Columns: Actual Classes


           0   1   2   3  total
cluster                        
0          1  82  34   4    121
1        171   2  17  27    217
2          0   6   3   0      9
3        580   1   0  72    653


##4.5(B)
###K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution
It follows the following steps:
- Generate Centroids by using helper function described above
- Write centroids into file
- Run MapReduce job with an upper limit of 1000 iterations
- Prints number of members of each cluster after each iteration
- Stops after the stopping criteria (described above) threshold of 0.001 is breached

In [67]:
# MapReduce for perturbed centroids with K=2

import imp
from itertools import chain
import kmeans_twitter
imp.reload(kmeans_twitter)

input_file_name = 'topUsers_Apr-Jul_2014_1000-words.txt'
mr_job = kmeans_twitter.kmeans(args=[input_file_name])

# Getting Centroids
num_centroids = 2
centroids = get_perturbed_centroids(Dataset_Centroid(), num_centroids)
print "Seed Centroids:"
print centroids

# Write Centroid to file
write_to_centroids_to_file(centroids)

# Running MapReduce twice with overall iteration limit set to 1000
for i in range(1000):
    print ">> Iteration:", i+1
    print ">> (Assigned Cluster, Number of Obserations)"
    old_centroids = read_centroids_file()
    # Running MapReduce job
    with mr_job.make_runner() as runner:
        runner.run()
        # Capturing MapReduce job output and printing to console
        for line in runner.stream_output():
            print mr_job.parse_output_line(line)
    new_centroids = read_centroids_file()
    if stop_criterion(old_centroids, new_centroids, 0.001):
        print "Stopping Criteria Reached"
        break

Seed Centroids:
[[  4.01222448e-02   3.21598335e-02   2.16428587e-02 ...,   1.27805138e-04
    2.33599521e-04   3.96367664e-04]
 [  4.03233081e-02   3.21045269e-02   2.15714409e-02 ...,   8.73486939e-05
    2.01127812e-04   4.43482525e-04]]
>> Iteration: 1
>> (Assigned Cluster, Number of Obserations)
(0, 839)
(1, 161)
>> Iteration: 2
>> (Assigned Cluster, Number of Obserations)
(0, 866)
(1, 134)
>> Iteration: 3
>> (Assigned Cluster, Number of Obserations)
(0, 869)
(1, 131)
>> Iteration: 4
>> (Assigned Cluster, Number of Obserations)
(0, 869)
(1, 131)
Stopping Criteria Reached


## The code below generates the class break-up per cluster
The rows are the clusters as assigned by Kmeans.  
The columns are the number of observations per label

In [68]:
# Generating summary results for perturbed random centroids for k=2
centroids = read_final_centroids()
results = results_data_structure()

# Looping through Input data
for c, d in input_data_generator():
    assigned_centroid = closest_centroid(centroids, d)
    results.loc[assigned_centroid, 'total'] += 1
    results.loc[assigned_centroid, int(c)] += 1

print ">> Cluster Summary"
print ">> Rows: Assigned Cluster; Columns: Actual Classes"
print "\n"
print results

>> Cluster Summary
>> Rows: Assigned Cluster; Columns: Actual Classes


           0   1   2   3  total
cluster                        
0        751   3  16  99    869
1          1  88  38   4    131
2          0   0   0   0      0
3          0   0   0   0      0


##4.5(C)
###K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution
It follows the following steps:
- Generate Centroids by using helper function described above
- Write centroids into file
- Run MapReduce job with an upper limit of 1000 iterations
- Prints number of members of each cluster after each iteration
- Stops after the stopping criteria (described above) threshold of 0.001 is breached

In [69]:
# MapReduce for perturbed centroids with K=4

import imp
from itertools import chain
import kmeans_twitter
imp.reload(kmeans_twitter)

input_file_name = 'topUsers_Apr-Jul_2014_1000-words.txt'
mr_job = kmeans_twitter.kmeans(args=[input_file_name])

# Getting Centroids
num_centroids = 4
centroids = get_perturbed_centroids(Dataset_Centroid(), num_centroids)
print "Seed Centroids:"
print centroids

# Write Centroid to file
write_to_centroids_to_file(centroids)

# Running MapReduce twice with overall iteration limit set to 1000
for i in range(1000):
    print ">> Iteration:", i+1
    print ">> (Assigned Cluster, Number of Obserations)"
    old_centroids = read_centroids_file()
    # Running MapReduce job
    with mr_job.make_runner() as runner:
        runner.run()
        # Capturing MapReduce job output and printing to console
        for line in runner.stream_output():
            print mr_job.parse_output_line(line)
    new_centroids = read_centroids_file()
    if stop_criterion(old_centroids, new_centroids, 0.001):
        print "Stopping Criteria Reached"
        break

Seed Centroids:
[[ 0.04019582  0.03221462  0.02142127 ...,  0.00023767  0.00024372
   0.00020157]
 [ 0.04028157  0.03220953  0.02162268 ...,  0.00020268  0.00011471
   0.00025136]
 [ 0.04037788  0.03207314  0.02147636 ...,  0.00014003  0.00020488
   0.0001804 ]
 [ 0.04035386  0.03214002  0.02142946 ...,  0.00014809  0.00023687
   0.00027616]]
>> Iteration: 1
>> (Assigned Cluster, Number of Obserations)
(0, 347)
(1, 456)
(2, 134)
(3, 63)
>> Iteration: 2
>> (Assigned Cluster, Number of Obserations)
(0, 306)
(1, 539)
(2, 64)
(3, 91)
>> Iteration: 3
>> (Assigned Cluster, Number of Obserations)
(0, 264)
(1, 602)
(2, 53)
(3, 81)
>> Iteration: 4
>> (Assigned Cluster, Number of Obserations)
(0, 212)
(1, 655)
(2, 53)
(3, 80)
>> Iteration: 5
>> (Assigned Cluster, Number of Obserations)
(0, 196)
(1, 671)
(2, 53)
(3, 80)
>> Iteration: 6
>> (Assigned Cluster, Number of Obserations)
(0, 182)
(1, 685)
(2, 53)
(3, 80)
>> Iteration: 7
>> (Assigned Cluster, Number of Obserations)
(0, 171)
(1, 696)
(2, 5

## The code below generates the class break-up per cluster
The rows are the clusters as assigned by Kmeans.  
The columns are the number of observations per label

In [70]:
# Generating summary results for perturbed random centroids for k=4
centroids = read_final_centroids()
results = results_data_structure()

# Looping through Input data
for c, d in input_data_generator():
    assigned_centroid = closest_centroid(centroids, d)
    results.loc[assigned_centroid, 'total'] += 1
    results.loc[assigned_centroid, int(c)] += 1

print ">> Cluster Summary"
print ">> Rows: Assigned Cluster; Columns: Actual Classes"
print "\n"
print results

>> Cluster Summary
>> Rows: Assigned Cluster; Columns: Actual Classes


           0   1   2   3  total
cluster                        
0         93   2  14  57    166
1        658   1   0  42    701
2          0  51   2   0     53
3          1  37  38   4     80


##4.5(D)
###K=4 "trained" centroids, determined by the sums across the classes
It follows the following steps:
- Generate Centroids by using helper function described above
- Write centroids into file
- Run MapReduce job with an upper limit of 1000 iterations
- Prints number of members of each cluster after each iteration
- Stops after the stopping criteria (described above) threshold of 0.001 is breached

In [71]:
# MapReduce for trainined centroids

import imp
from itertools import chain
import kmeans_twitter
imp.reload(kmeans_twitter)

input_file_name = 'topUsers_Apr-Jul_2014_1000-words.txt'
mr_job = kmeans_twitter.kmeans(args=[input_file_name])

# Getting Centroids
centroids = get_trainined_centroids()
print "Seed Centroids:"
print centroids

# Write Centroid to file
write_to_centroids_to_file(centroids)

# Running MapReduce twice with overall iteration limit set to 1000
for i in range(1000):
    print ">> Iteration:", i+1
    print ">> (Assigned Cluster, Number of Obserations)"
    old_centroids = read_centroids_file()
    # Running MapReduce job
    with mr_job.make_runner() as runner:
        runner.run()
        # Capturing MapReduce job output and printing to console
        for line in runner.stream_output():
            print mr_job.parse_output_line(line)
    new_centroids = read_centroids_file()
    if stop_criterion(old_centroids, new_centroids, 0.001):
        print "Stopping Criteria Reached"
        break

Seed Centroids:
[[  1.28071303e-02   4.74992198e-02   2.60213372e-02 ...,   0.00000000e+00
    3.21055688e-04   3.11719199e-04]
 [  1.08473360e-01   2.49464165e-03   1.02660496e-02 ...,   0.00000000e+00
    9.71699557e-06   2.02218556e-05]
 [  6.54587536e-02   4.55253175e-03   2.03868542e-02 ...,   1.26701038e-03
    2.13373254e-06   6.40119762e-06]
 [  3.15310774e-02   4.23890795e-02   1.81846417e-02 ...,   0.00000000e+00
    7.84106068e-05   1.05735515e-04]]
>> Iteration: 1
>> (Assigned Cluster, Number of Obserations)
(0, 787)
(1, 61)
(2, 82)
(3, 70)
>> Iteration: 2
>> (Assigned Cluster, Number of Obserations)
(0, 796)
(1, 54)
(2, 85)
(3, 65)
>> Iteration: 3
>> (Assigned Cluster, Number of Obserations)
(0, 802)
(1, 51)
(2, 84)
(3, 63)
>> Iteration: 4
>> (Assigned Cluster, Number of Obserations)
(0, 804)
(1, 51)
(2, 82)
(3, 63)
>> Iteration: 5
>> (Assigned Cluster, Number of Obserations)
(0, 804)
(1, 51)
(2, 82)
(3, 63)
Stopping Criteria Reached


## The code below generates the class break-up per cluster
The rows are the clusters as assigned by Kmeans.  
The columns are the number of observations per label

In [72]:
# Generating summary results for trained centroids
centroids = read_final_centroids()
results = results_data_structure()

# Looping through Input data
for c, d in input_data_generator():
    assigned_centroid = closest_centroid(centroids, d)
    results.loc[assigned_centroid, 'total'] += 1
    results.loc[assigned_centroid, int(c)] += 1

print ">> Cluster Summary"
print ">> Rows: Assigned Cluster; Columns: Actual Classes"
print "\n"
print results

>> Cluster Summary
>> Rows: Assigned Cluster; Columns: Actual Classes


           0   1   2   3  total
cluster                        
0        749   3  14  38    804
1          0  51   0   0     51
2          1  37  40   4     82
3          2   0   0  61     63
